In [1]:
import numpy as np
import mc_lj_potential
import pytest
import sys
import os
class Box:
    def __init__(self, box_length, coordinates=None):
        self.box_length=box_length
        self.coordinates=coordinates
    
    def wrap(self, coordinates,box_length):
        """
        This is for wraping all particles in the box, updating the coordinates.

        Parameters
        ----------
        coordinates : np.array
        

        """
        if (coordinates is not None):
            self.coordinates = self.coordinates - self.box_length*round(self.coordinates/self.box_length)
            
    def minimum_image_distance(self, r_i, r_j, box_length):
        rij = r_i - r_j
        rij = rij - self.box_length * np.round(rij / self.box_length)
        rij2 = np.dot(rij, rij)
        return rij2
        
    @property
    def volume(self):
        return self.box_length**3
    
    @property
    def num_particles(self):
        if (isinstance(self.coordinates,type(None))):
            return None
        else:
            return len(self.coordinates)
            
class MCState:
    def __init__(self,box1,cutoff):
        self.box1=box1
        self.cutoff=cutoff
#        self.total_pair_energy=0.0
#        self.particle_energy=0.0
        self.tail_correction=0.0
        self.unit_energy=0.0
    
    def calculate_total_pair_energy(self):
        self.total_pair_energy=0.0
        particle_count = len(self.box1.coordinates)
        for i_particle in range(particle_count):
            for j_particle in range(i_particle):
                r_i = self.box1.coordinates[i_particle]
                r_j = self.box1.coordinates[j_particle]
                rij2 = self.box1.minimum_image_distance(r_i, r_j, self.box1.box_length)
                if rij2 < self.cutoff**2:
                    self.total_pair_energy += self.lennard_jones_potential(rij2)
        return self.total_pair_energy
        
    def calculate_tail_correction(self):
        sig_by_cutoff3 = np.power(1.0 / self.cutoff, 3)
        sig_by_cutoff9 = np.power(sig_by_cutoff3, 3)
        self.tail_correction = sig_by_cutoff9 - 3.0 * sig_by_cutoff3
        self.tail_correction *= (8.0 / 9.0) * np.pi * self.box1.num_particles * self.box1.num_particles/ self.box1.volume
        return self.tail_correction

    def calculate_unit_energy(self):
        self.unit_energy=(self.total_pair_energy + self.tail_correction)/len(self.box1.coordinates)
        return self.unit_energy
    
    def get_particle_energy(self, i_particle):
        self.particle_energy = 0.0
        i_position = self.box1.coordinates[i_particle]
        particle_count = len(self.box1.coordinates)
        for j_particle in range(particle_count):
            if i_particle != j_particle:
                j_position = self.box1.coordinates[j_particle]
                rij2 = self.box1.minimum_image_distance(i_position, j_position, self.box1.box_length)
                if rij2 < self.cutoff**2:
                    e_pair = self.lennard_jones_potential(rij2) 
                    self.particle_energy += e_pair
        return self.particle_energy
    
    def lennard_jones_potential(self, rij2):
        sig_by_r6 = np.power(1 / rij2, 3)
        sig_by_r12 = np.power(sig_by_r6, 2)
        return 4.0 * (sig_by_r12  - sig_by_r6)
    
def generate_initial_state(method = 'random', file_name = None, num_particles = None, box_length = None):
    """ Generates initial state of the system.
​
     Generates the initial coordinates of all the atoms in the simulation box. If the method is random, the atoms are assigned a random set of coordinates.
     If method is File, coordinates are loaded from a file.
​
    Parameters
    ----------
    method : string. Either 'random' or 'File'.
        Flag which is either set to random or File depending on whether we need random coordinates or load coordinates from a file.
    file_name :  string. Default is None.
        File name to load coordinates from if method is File.
    num_particles : integer. Default is none.
        Number of particles in the simulation box.
    box_length : float. Default is None
        Side of cubic simulation box.
    
    Returns
    -------
    coordinates : np.array(num_particles,3)
        A numpy array with the x,y and z coordinates of each atom in the simulation box.
    """
    if method is 'random':
        coordinates = 0.5 - np.random.rand(num_particles, 3) * box_length
    
    elif method is 'file':
        coordinates = np.loadtxt(file_name, skiprows = 2, usecols=(1, 2, 3))
    return coordinates

def accept_or_reject(delta_e, beta):
    """
    Accepts or rejects a move based on the energy difference between initial and updated state along with system temperature
    
    Parameters
    ----------
    delta_e : float
        Energy difference between initial and updated state of the system.
    beta : float
        Inverse reduced temperature, a general constant in canonical ensemble.
​
    Returns
    -------
    accept : boolean
        If true, trial move is accepted, else it is rejected.
​
    """
    if delta_e < 0.0:
        accept = True
    
    else:
        random_number = np.random.rand(1)
        p_acc = np.exp(-beta * delta_e)
        if random_number < p_acc:
            accept = True
        else:
            accept = False
    return accept

def adjust_displacement(n_trials, n_accept, max_displacement):
    """Adjusts the maximum value allowed for a displacement move.
    
    This function adjusts the maximum displacement to obtain a suitable acceptance \
    of trial moves. That is, when the acceptance is too high, the maximum \
    displacement is increased and when the acceptance is too low, the \
    maximum displacement is decreased.
    
    Parameters
    ----------
    n_trials : integer
        Number of trials that have been performed when the funtction is called.
    n_accept: integer
        Number of current accepted trials when the function is called.
    max_displacement: float
        Maximum displacement allowed for any step in the simulation.
    
    Returns
    -------
    n_trials: int
        Number of trials. Updated to zero if maximum displacement is updated.
    n_accept: int
        Number of trials. Updated to zero if maximum displacement is updated.
    max_displacemnt: float
        Maximum displacement allowed for any step in the simulation. 
    """
    acc_rate = float(n_accept)/float(n_trials)
    if (acc_rate < 0.38):
        max_displacement *= 0.8
    
    elif (acc_rate > 0.42):
        max_displacement *= 1.2

    n_trials = 0
    n_accept = 0
    return max_displacement, n_trials, n_accept

In [2]:
coordinates = mc_lj_potential.generate_initial_state(method = "random", box_length =10.0, num_particles = 100)
box_length = 10.0
cutoff = 3.0
box = mc_lj_potential.Box(box_length=box_length, coordinates=coordinates)
mcs = mc_lj_potential.MCState(box, cutoff = cutoff)

In [7]:
mcs.calculate_total_pair_energy()

69248.20756514888

In [9]:
mcs.calculate_tail_correction()

-3.101388808502446

In [3]:
mcs.calculate_unit_energy()

AttributeError: 'MCState' object has no attribute 'total_pair_energy'